In [4]:
import os 
import glob
from pathlib import Path
import pandas as pd 
import numpy as np 
import ast
import json 

In [2]:
map_dti = pd.read_csv('/working/lupolab/julia/tcia_analysis/code/dti_seg_map.csv', index_col = 0)

In [3]:
map_dti.head()

,cohort,dti,exam,patient_id,segmentation_name
0,TCGA-LGG-nifti,"['400-DTI_2.6mm_WH-19253.nii.gz', '4-DTI_2.6mm...",08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,TCGA-DU-5853,11-AX_T1_POST_GD_FLAIR-81300a_n
1,TCGA-LGG-nifti,['205-dadc-52244.nii.gz'],06-01-2000-MRI_BRAIN_per_R-97608,TCGA-CS-6186,1001-T1_SE_POST-94212a_n
2,TCGA-LGG-nifti,"['12-ep2ddiffmddw20p2_wipADC-35015.nii.gz', '1...",04-23-2004-NR_MRI_BRAIN_WWO-08141,TCGA-FG-A6J1,20-T1fl2dAx__Gd-43129a_n
3,TCGA-LGG-nifti,"['601-DTI_ASSET-84919.nii.gz', '600-DTI_ASSET-...",06-18-1989-MRI_BRAIN_WWO_CONTRAS-16160,TCGA-DU-7015,10-AX_T1_POST_GD_FLAIR-19430a_n
4,TCGA-LGG-nifti,"['501-DTI_2.6mm_WH-21034.nii.gz', '5-DTI_2.6mm...",01-11-1997-MRI_BRAIN_WWO_CONTRAST-98366,TCGA-DU-8164,10-AX_T1_POST_GD_FLAIR-02819a_n


## Here we're going to figure out which came with preprocessed ADC: 

In [8]:
map_dti['adc'] = ['' for x in range(map_dti.shape[0])]

In [22]:
for idx, row in map_dti.iterrows(): 
    list_of_dti = ast.literal_eval(row['dti'])
    adc_list = [x for x in list_of_dti if 'adc' in x.lower() or 'apparent' in x.lower() or 'coeff' in x.lower() or 'average' in x.lower()]
    if adc_list: 
        row['adc'] = adc_list
    else: 
        row['adc'] = float("NaN")

In [25]:
map_dti['adc'].isnull().value_counts()

False    70
True     68
Name: adc, dtype: int64

In [35]:
map_dti.head()

,cohort,dti,exam,patient_id,segmentation_name,adc
0,TCGA-LGG-nifti,"['400-DTI_2.6mm_WH-19253.nii.gz', '4-DTI_2.6mm...",08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,TCGA-DU-5853,11-AX_T1_POST_GD_FLAIR-81300a_n,NaN
1,TCGA-LGG-nifti,['205-dadc-52244.nii.gz'],06-01-2000-MRI_BRAIN_per_R-97608,TCGA-CS-6186,1001-T1_SE_POST-94212a_n,[205-dadc-52244.nii.gz]
2,TCGA-LGG-nifti,"['12-ep2ddiffmddw20p2_wipADC-35015.nii.gz', '1...",04-23-2004-NR_MRI_BRAIN_WWO-08141,TCGA-FG-A6J1,20-T1fl2dAx__Gd-43129a_n,"[12-ep2ddiffmddw20p2_wipADC-35015.nii.gz, 10-D..."
3,TCGA-LGG-nifti,"['601-DTI_ASSET-84919.nii.gz', '600-DTI_ASSET-...",06-18-1989-MRI_BRAIN_WWO_CONTRAS-16160,TCGA-DU-7015,10-AX_T1_POST_GD_FLAIR-19430a_n,NaN
4,TCGA-LGG-nifti,"['501-DTI_2.6mm_WH-21034.nii.gz', '5-DTI_2.6mm...",01-11-1997-MRI_BRAIN_WWO_CONTRAST-98366,TCGA-DU-8164,10-AX_T1_POST_GD_FLAIR-02819a_n,NaN


## Here are the manually processed DTI: 

In [54]:
for idx, row in map_dti.iterrows(): 
    
    path = '/working/lupolab/julia/tcia_analysis/datasets/'+'-'.join(row['cohort'].split('-')[:-1])+'/'+row['patient_id']+'/'+row['exam']
    series_in_exam = os.listdir(path)
    print(series_in_exam)
    if 'diffusion_b=1000' in series_in_exam : 
        row['adc'] = 'diffusion_b=1000'
    elif 'diffusion_b=1200' in series_in_exam: 
        row['adc'] = 'diffusion_b=1200'
    

['3-SAG_T1_FLAIR-94337', '4-DTI_2.6mm_WH-81596', '12-COR__T1_POST_GD_FLAIR-32689', '401-DTI_2.6mm_WH-97906', 'final_dti_subset.csv', '7-AX_T2_FR-FSE_RF2_150-55295', '10-3D_DCE_DYNAMIC_PERF-61302', '11-AX_T1_POST_GD_FLAIR-81300', '6-AXIAL_FLAIR-84159', '5-AXIAL_DIFFUSION-35619', 'diffusion_b=1000', '8-AX_T1_pre_gd-14019', '400-DTI_2.6mm_WH-19253', '9-3D_DCE_T1_MAP-73753', 'Logfile', '402-DTI_2.6mm_WH-69647']
['1101-STRYKER-27064', '605-MRA-59702', '701-AX_PD-23493', '201-DIFFUSION-10198', '301-T1_SAG_SE-41348', '101-SurveyMST-18717', '1201-T1_COR_POST-35170', '801-T2WFFE_bleed-67839', '1301-T1_SAG_POST-08017', '603-RT___MRA-44425', '602-3D_MC__MRA-83030', '901-T1_SE_PRE-16483', '401-T2_AX_FLAIR-72445', '501-T2-WHOLE_BRAIN-32557', '203-eB_0-99844', '205-dadc-52244', '204-eB1000I-81870', '601-3D_MC__MRA-29688', '202-Reg_-_DIFFUSION_SENSE-00561', '1001-T1_SE_POST-94212', '604-LT___MRA-27206']
['13-ep2ddiffmddw20p2_wipTRACEW-63745', '11-ep2ddiffmddw20p2_wip-78279', '15-ep2ddiffmddw20p2_wipC

['9-MPRAGE_SAG__IPAT-86123', '2-MRHR_T2_AX-69038', '8-MRHR_DIFF_IPATADC-99808', '10-T1_AX-92227', '1-MRHR_SCOUT-74219', '13-MRHR_T1_AX_POST_GAD-66565', '11-MRHR_T1_AX_POST_GAD-48215', '102-MPR_SAG-29359', '12-MRHR_T1_COR_POST_GAD-78825', '5-MRHR_FLAIR_AX-17349', '4-MRHR_FLAIR_AX-42830', '7-MRHR_DIFF_IPAT-05020', '100-MPR_AX-95760', '101-MPR_COR-13800', '14-MRHR_T1_SAG_POST_GAD-79916']
['3-2D_EPI_DWI_ASSET-62914', '7-T2Prop-24727', '12-COR_T1_SEC-59798', '11-C_Ax_T1_MP_SPGR-28808', '10-SAG_T1_SEC-31391', '5-Ax_T2_Flair-75934', '300-Exponential_Apparent_Diffusion_Coefficient-99753', '9-Ax_T1_MP_SPGR-99964', '8-Cor_2D_T2GRE-97249', '301-Apparent_Diffusion_Coefficient_mm2s-82769', '4-Sag_T2_FSE-04153', '6-Ax_T1_SE-88546']
['9-8hrbrainAx2DDWIProp-96105', '900-PROP_ADC-27269', '5-AX_T1-38562', '11-COR_T1C-21635', '301-Apparent_Diffusion_Coefficient_mms-86725', '901-PROP_eADC-40494', '8-Prop_T2_TRF-14460', '7-COR_GRE_T2-97585', '10-SAG_T1C-56341', '12-CAX_3D_SPGR-92286', '3-Ax_DW_DW-EPI_S_b10

['4-AX_3D_SPGR-08228', '8-Ax_Flair_irFSE_H-30437', '901-Apparent_Diffusion_Coefficient_ms-81614', '5-COR_GRE_T2-36967', '1-3-pl_T2_FGRE_S-04369', '11-SAG_T1C-50563', '6-SAG_T1-72398', '9-Ax_DW_DW-EPI_S_b1000T2-84695', '10-AX_3D_SPGRC-88568', '900-Exponential_Apparent_Diffusion_Coefficient-01613', '7-Ax_T2_FSE_H-78773', '12-COR_T1C-08484']
['501-DIFFUSION-08793', '1201-T1_COR_SE-17084', '1003-SAG_MPR_ISO1MM-83617', '401-SAG_T1-02443', '1001-T1W3DSTRYKER-84137', '502-eBo-00599', '1301-T1_SAG_SE-08339', '601-T2WFFE-90138', '101-Survey-94672', '1002-COR_MPR_ISO1MM-45390', '503-eB1000i-73091', '801-AXIAL__T2-43433', '504-dADC_MAP-93398', '1101-T1_AX_SE_FS_POST-08997', '701-T2_AX_FLAIR-08756', '901-T1_AX__SE-78879']
['12-COR_T1C-68868', '802-PROP_COMB-11163', '801-PROP_eADC-93184', '400-Exponential_Apparent_Diffusion_Coefficient-95285', '401-Apparent_Diffusion_Coefficient_mms-85810', '3-SAG_T1-99851', '9-Prop_T2_TRF-28138', '4-Ax_DW_DW-EPI_S_b1000T2-41290', '800-PROP_ADC-80124', '7-COR_GRE_T

['702-DTI_ASSET-69796', '11-AX_T1_POST_GD_FLAIR-22225', '700-DTI_ASSET-17883', '12-COR_T1_POST_GD_FLAIR-40179', '701-DTI_ASSET-07155', '13-AX_T1_POST_GD_FLAIR-16002']
['8-AX_T1_PostFIL-73586', '10-AX_T2_FS-03857', '7-AX_T2_FS-39199', '13-AX_DWI_WIP-91895', '12-SAG_MP-RAGE_Post-30819', '14-AX_DWI_WIPADC-88397', '11-AX_FLAIR-01197', '100-CORONAL_MPR-68548', '4-AX_FLAIR-62483', '5-AX_T2_GREFIL-81946', '16-SAG_3D_DIX_Postopp-35264', '9-AX_T1_PostFIL-17853', '17-SAG_3D_DIX_PostW-83305', '6-AX_T1-26978', '15-SAG_3D_DIX_Postin-51317']
['4-Ax__DW-EPI-54432', '6-COR_T1_FSE_C-21588', '2-Ax_T2_FSE_H-23335', '672-Apparent_Diffusion_Coefficien-26049', '3-Ax_Flair_irFSE_H-76518', '1-Sag_T1_FSE_S-74015', '7-COR_T1_FSE_C-24007', '5-AX_T1_FSE_HC-14079', '8-AX_T1_FSE_HC-10944']
['1101-T1_AX_SE_FS_POST-29108', '1003-SAG_MPR_ISO1MM-87549', '1001-T1W3DSTRYKER-96114', '403-eB1000i-48518', '101-Survey-37713', '901-T1_AX__SE-25444', '404-dADC_MAP-42057', '1201-T1_COR_SE-35572', '701-AXIAL__T2-73118', '402-eBo

['601-BRAIN_WWO_T2_HEMOSID-56262', '501-BRAIN_WWO_T2_GRASE-38485', '301-BRAIN_WWO_T1_SAG-44016', '1001-BRAIN_WWO_COR_POST-97459', '901-BRAIN_WWO_T1_POST-50983', '702-BRAIN_WWO_SS_DWI-07290', '401-BRAIN_WWO_FLAIR-32434', '701-BRAIN_WWO_SS_DWI-12278', '801-BRAIN_WWO_T1_PRE-14606']
['8-FLAIR_AXIAL-14764', '7-T2_TSE_COR-07685', '3-T1_SAG-50719', '11-T1_COR_C-72346', '4-ep2ddifforthbasic_PAT2-23658', '1-localizer-02129', '9-T1_AXIAL-25337', '10-T1_AXIAL__POST-97398', '2-localizer-63884', '5-ADC-57186', '6-T2_TSE_AXIAL-58552']
['2-localizer-24375', '3-T2_AX-62164', '10-T1_COR_Gd-35380', '11-T1_SAG_Gd-56476', '6-DIFFUSIONADC-19007', '9-T1_AXIAL_Gd-31768', '7-FLAIR_AX-90386', '5-DIFFUSION-40367', '8-TIR_T1_AX-02076']
['7-Ax_DWI_1000b_ASSET-93286', '700-Ax_DWI_1000b_ASSET-51482', '701-Ax_DWI_1000b_ASSET-53063', '22-nordicICE_HFH_-_MTT_map_-Leakage_corrected-217.0', '5-Ax_T2_FLAIR-33749', '10-3d_T1_FSPGR__CONT-32567', '21-nordicICE_HFH_-_rBF_map_-Leakage_corrected-624.0', '6-ASSET_Cal-35485', '2

['6-PERFUSION_15_sec_inject.delay-13363', '814-Apparent_Diffusion_Coefficien-13024', '5-3D_FSE_AX-42060', '4-AXIAL_DIFFUSION-70965', '20-nordicICE_HFH_-_rBV_map_-Leakage_corrected-898.0', '3-AX_FLAIR-52108', '2-FSPGR_SAG-82544', '815-Attenuation_Coefficient_Map-20564', '22-nordicICE_HFH_-_MTT_map_-Leakage_corrected-300.0', '7-3D_SPGR_AX-36833', '816-Combined_Image-58868', '1-3-PLANE_LOC-18653', '601-Negative_Enhancement_Integral-65961', '21-nordicICE_HFH_-_rBF_map_-Leakage_corrected-754.0']
['9-DTIColFA-98462', '14-MRHR_T1_COR_POST_GAD-57836', 'Logfile', '1-MRHR_SCOUT-41314', '11-MRHR_T1_AX-84384', '7-DTIADC-43987', '3-MRHR_FLAIR_AX-17941', '2-MRHR_T2_AX-61279', '13-MRHR_T1_AX_POST_GAD-93013', '15-MRHR_T1_SAG_POST_GAD-28393', '6-DTI-72659', '5-MRHR_DIFF_IPATADC-07518', '4-MRHR_DIFF_IPAT-29471', '8-DTIFA-00791', '12-MP_RAGE_AXIAL_VOLUMETRIC-21079', 'diffusion_b=']
['20-nordicICE_HFH_-_rBV_map_-Leakage_corrected-306.0', '8-MRSI_-VSS-88935', '12-FSE_Tractography_Stealth-12040', '7-AX_FLAI

In [56]:
map_dti.adc.isnull().value_counts()

False    100
True      38
Name: adc, dtype: int64

In [74]:
map_dti_to_align = map_dti.loc[~map_dti.adc.isnull()]
map_dti_to_align.shape

(100, 6)

In [88]:
map_dti_to_align.head(10)

,cohort,dti,exam,patient_id,segmentation_name,adc
0,TCGA-LGG-nifti,"['400-DTI_2.6mm_WH-19253.nii.gz', '4-DTI_2.6mm...",08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,TCGA-DU-5853,11-AX_T1_POST_GD_FLAIR-81300a_n,diffusion_b=1000
1,TCGA-LGG-nifti,['205-dadc-52244.nii.gz'],06-01-2000-MRI_BRAIN_per_R-97608,TCGA-CS-6186,1001-T1_SE_POST-94212a_n,[205-dadc-52244.nii.gz]
2,TCGA-LGG-nifti,"['12-ep2ddiffmddw20p2_wipADC-35015.nii.gz', '1...",04-23-2004-NR_MRI_BRAIN_WWO-08141,TCGA-FG-A6J1,20-T1fl2dAx__Gd-43129a_n,"[12-ep2ddiffmddw20p2_wipADC-35015.nii.gz, 10-D..."
3,TCGA-LGG-nifti,"['601-DTI_ASSET-84919.nii.gz', '600-DTI_ASSET-...",06-18-1989-MRI_BRAIN_WWO_CONTRAS-16160,TCGA-DU-7015,10-AX_T1_POST_GD_FLAIR-19430a_n,diffusion_b=1000
4,TCGA-LGG-nifti,"['501-DTI_2.6mm_WH-21034.nii.gz', '5-DTI_2.6mm...",01-11-1997-MRI_BRAIN_WWO_CONTRAST-98366,TCGA-DU-8164,10-AX_T1_POST_GD_FLAIR-02819a_n,diffusion_b=1000
5,TCGA-LGG-nifti,"['802-DTI_2.6mm_WH-35614.nii.gz', '600-Exponen...",07-26-1997-MRI_BRAIN_WWO_CONTRAST-17402,TCGA-DU-A5TR,10-AX_T1_POST_GD_FLAIR-23755a_n,[600-Exponential_Apparent_Diffusion_Coefficien...
6,TCGA-LGG-nifti,"['900-PROP_ADC-74032.nii.gz', '401-Apparent_Di...",10-28-1995-MRI_BRAIN_FOR_STEREOTACTIC_WWO_CONT...,TCGA-HT-7604,12-CAX_3D_SPGR-37353a_n,"[900-PROP_ADC-74032.nii.gz, 401-Apparent_Diffu..."
8,TCGA-LGG-nifti,"['6-DTI_2.6mm_WH-60904.nii.gz', '601-DTI_2.6mm...",11-19-1996-MRI_BRAIN_WWO_CONTRAST-27038,TCGA-DU-8163,12-AX_T1_POST_GD_FLAIR-07772a_n,diffusion_b=1000
9,TCGA-LGG-nifti,"['1100-DTI_ASSET-99068.nii.gz', '1101-DTI_ASSE...",12-20-1991-MRI_BRAIN_WWO_CONTRAST-98852,TCGA-DU-7018,13-AX_T1_POST_GD_FLAIR-47091a_n,diffusion_b=1000
10,TCGA-LGG-nifti,['4-ep2ddiff3scantraceADC-62120.nii.gz'],09-17-2001-MRI_BRAIN__W_WO_CONT-92660,TCGA-EZ-7265A,10-T1AXMPRSEL3D-83348a_n,[4-ep2ddiff3scantraceADC-62120.nii.gz]


In [119]:
# map_dti_to_align.to_csv('/working/lupolab/julia/tcia_analysis/code/aligned_dti.csv', index = False)

## Register the DTI 

In [58]:
os.chdir('/working/lupolab/julia/tcia_analysis/code')

In [64]:
usable_patient_gbm = pd.read_csv('usable_patient_gbm.csv', index_col = 0)

In [65]:
usable_patient_lgg = pd.read_csv('usable_patient_lgg.csv', index_col = 0)

In [72]:
usable_patients = pd.concat([usable_patient_gbm, usable_patient_lgg], ignore_index=True)

In [76]:
usable_patients.head(1)

,ground_truth,image,sd,patient_id,exam_id,fixed_vol
0,T1C,/working/lupolab/julia/tcia_analysis/datasets/...,701-BRAIN_WWO_T1_COR_POST-08207,TCGA-14-0865,06-04-1995-MRI_BRAIN_W_WOUT_CONTRAST-31488,/working/lupolab/julia/tcia_analysis/datasets/...


Need to find the fixed volume using patient id/ exam/ & then just create command to do this with diffusion: 

`BRAINSFit --fixedVolume $fixed_vol.nii.gz --movingVolume $moving_vol.nii.gz --outputVolume $output_vol.nii.gz --outputTransform $transform_output.tfm  --transformType Rigid`

In [115]:
# map_dti_to_align['aligned_dti'] = ['' for x in range(map_dti_to_align.shape[0])]

In [118]:
map_dti_to_align.head()

,cohort,dti,exam,patient_id,segmentation_name,adc,aligned_dti
0,TCGA-LGG-nifti,"['400-DTI_2.6mm_WH-19253.nii.gz', '4-DTI_2.6mm...",08-23-1995-MRI_BRAIN_WWO_CONTRAST-78436,TCGA-DU-5853,11-AX_T1_POST_GD_FLAIR-81300a_n,diffusion_b=1000,/working/lupolab/julia/tcia_analysis/datasets/...
1,TCGA-LGG-nifti,['205-dadc-52244.nii.gz'],06-01-2000-MRI_BRAIN_per_R-97608,TCGA-CS-6186,1001-T1_SE_POST-94212a_n,[205-dadc-52244.nii.gz],[/working/lupolab/julia/tcia_analysis/datasets...
2,TCGA-LGG-nifti,"['12-ep2ddiffmddw20p2_wipADC-35015.nii.gz', '1...",04-23-2004-NR_MRI_BRAIN_WWO-08141,TCGA-FG-A6J1,20-T1fl2dAx__Gd-43129a_n,"[12-ep2ddiffmddw20p2_wipADC-35015.nii.gz, 10-D...",[/working/lupolab/julia/tcia_analysis/datasets...
3,TCGA-LGG-nifti,"['601-DTI_ASSET-84919.nii.gz', '600-DTI_ASSET-...",06-18-1989-MRI_BRAIN_WWO_CONTRAS-16160,TCGA-DU-7015,10-AX_T1_POST_GD_FLAIR-19430a_n,diffusion_b=1000,/working/lupolab/julia/tcia_analysis/datasets/...
4,TCGA-LGG-nifti,"['501-DTI_2.6mm_WH-21034.nii.gz', '5-DTI_2.6mm...",01-11-1997-MRI_BRAIN_WWO_CONTRAST-98366,TCGA-DU-8164,10-AX_T1_POST_GD_FLAIR-02819a_n,diffusion_b=1000,/working/lupolab/julia/tcia_analysis/datasets/...


In [117]:
commands = []
for idx, row in map_dti_to_align.iterrows(): 
    patient_id = row['patient_id']
    fixed_vol_df = usable_patients.loc[usable_patients.patient_id == patient_id]['fixed_vol'].reset_index()
    fixed_vol = (fixed_vol_df['fixed_vol'][0])
    root_dir = '/working/lupolab/julia/tcia_analysis/datasets/'
    cohort = row['cohort']
    exam = row['exam']
    if 'diffu' in row['adc']: 
        moving_vol = str(Path(root_dir, cohort, patient_id, exam, patient_id+'_adc.nii.gz'))
        output_vol = str(Path(root_dir, cohort, patient_id, exam, patient_id+'_adca.nii.gz'))
        command_align = 'BRAINSFit --fixedVolume '+fixed_vol+' --movingVolume '+moving_vol+' --outputVolume '+output_vol+' --transformType Rigid'
        commands.append(command_align)
        row['aligned_dti'] = output_vol
    else: 
#         print(type(row['adc']))
#         list_adc = ast.literal_eval(row['adc'])
        output_vols = []
        for series in row['adc']: 
            moving_vol = str(Path(root_dir, cohort, patient_id, exam, series))
            output_vol = str(Path(root_dir, cohort, patient_id, exam, '.'.join(series.split('.')[:-2])+'a.nii.gz'))
            command_align = command_align = 'BRAINSFit --fixedVolume '+fixed_vol+' --movingVolume '+moving_vol+' --outputVolume '+output_vol+' --transformType Rigid'
            commands.append(command_align)
            output_vols.append(output_vol)
        row['aligned_dti'] = output_vols

In [105]:
os.chdir('/working/lupolab/julia/tcia_analysis/code/')

In [106]:
x = open('register_ADC.txt', 'w')

In [107]:
x.write('{')
x.write('\n')
for command in commands: 
    x.write(command)
    x.write('\n')
x.write('}')
x.close()

In [108]:
!grid_pipeline --scriptonly register_ADC.txt


In [109]:
# Run qsub register_ADC.txt_1.csh on Atom at Sun 3:33 pm Mar 1, 2020
